In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from timeit import default_timer

import paddle
from paddle.io import DataLoader, TensorDataset

paddle.seed(0)
np.random.seed(0)
paddle.set_device('gpu')

# Real-time design optimization 

During optimization, we start from initial design parameters
$(L_p = 100, x2 = -40, x3 = -30, h = 25.0) \mu m$, and update them using the BFGS algorithm to minimize the objective function ⟨Xup⟩ post-processed from the bacteria population predicted by Geo-FNO. When the optimization gets trapped in a local minimizer, the optimization restarts from an initial condition obtained by perturbing the recordedglobal minimizer with a random Gaussian noise sampled from $N(0, I)$. The proposed randomized BFGS algorithm guarantees the recorded-global minimizer monotonically decreases. 

In [ ]:
################################################################
# inverse optimization for 1d
################################################################
from catheter import *
model = FNO1d(64, 64, 100, input_channel=2, output_np=2001)
model.set_state_dict(paddle.load("/home/lunhao/Replication/AiScience/geofno-paddle/model/catheter_plain_length_model_1d1000.pdparams"))



# constraints   
#               60 < L_p < 250
#               x1 = -0.5L_p 
#               
#               15 < x3 - x1 < L_p/2
#             -L_p < x2      < 0
#               20 < h       < 30
# def transfer(theta):
def transfer(theta):
    
    L_p = 60 + (250 - 60)/(1 + paddle.exp(theta[0]))
    x1 = -0.5*L_p
    x3 = x1  + 15 + (L_p/2 - 15)/(1 + paddle.exp(theta[2]))
    x2 = -L_p  + L_p/(1 + paddle.exp(theta[1]))
    h = 20   + (10)/(1 + paddle.exp(theta[3]))
    return L_p, x1, x2, x3, h


def inv_transfer(L_p, x2, x3, h):
    x1 = -0.5*L_p
    theta = np.zeros(4)
    theta[0] = np.log( (250 - 60)/(L_p - 60) - 1 )
    theta[1] = np.log( L_p/(x2 + L_p) - 1 )
    theta[2] = np.log( (L_p/2 - 15)/(x3 - x1  - 15) - 1 )
    theta[3] = np.log( 10/(h - 20 ) - 1 )
    return theta

L_p, x2, x3, h = 100.0, -40.0, -30.0, 25.0
theta0 =  inv_transfer(L_p, x2, x3, h) 
print("initialize : ", theta0)


bacteria_pred = []
L_x = 500
N_s = 2001
xx_mask = (torch.linspace(1.0, 0, N_s) * (-L_x))

max_iter = 100


loss_min = paddle.tensor(np.inf)

loss_all = []
theta_min = np.copy(theta0)
theta_min_perturbed = np.copy(theta_min)


def saveplot(theta, out, savefig_name):
    bacteria_pred.append(out)
    L_p, x1, x2, x3, h = transfer(theta)
    mesh = x.detach().cpu().numpy()
    
    current_loss = (-torch.sum(torch.matmul(out, xx_mask))* L_x/N_s).item()
    
    print("L_p, x1, x2, x3, h  = ", L_p.item(), x1.item(), x2.item(), x3.item(), h.item())
    plt.figure(figsize=(4,4), dpi=1200)
    plt.title("Real-time design optimization")
    plt.plot(mesh[0, :, 0], mesh[0, :, 1], color="C1", label="Channel geometry")
    plt.plot(mesh[0, :, 0], 100-mesh[0, :, 1], color="C1")
    
    plt.plot(xx_mask.detach().cpu().numpy(), out.detach().cpu().numpy().T, "--*", color="C2", fillstyle='none', markevery=len(xx_mask)//10, label="Predicted bacteria distribution")
    
    plt.legend(loc="upper left")
    plt.ylim([-50,500])
    plt.xlabel(r"x")
    
    plt.text(-300,300,"Loss = "+r"$%3.1f$" %current_loss,\
          bbox={'facecolor':'white','alpha':1,'edgecolor':'none','pad':1})
    
    
    plt.tight_layout()
    plt.savefig(savefig_name)
    plt.show()
    
plot_id = 0

# first_figure
theta = torch.tensor(theta_min_perturbed.astype(np.float32) , requires_grad=True)
L_p, x1, x2, x3, h = transfer(theta)  
x, XC, YC = catheter_mesh_1d_total_length(L_x, L_p, x2, x3, h, N_s)
out = torch.exp(model(x).squeeze())
saveplot(theta, out, "movie/design_iter" + str(plot_id).zfill(4) + ".png")
plot_id += 1

for ep in range(epochs):
    theta = torch.tensor(theta_min_perturbed.astype(np.float32) , requires_grad=True)
    optimizer = torch.optim.LBFGS([theta], max_iter=max_iter, lr=1.0, line_search_fn="strong_wolfe")
    
    t1 = default_timer()
    def loss_closure():
        L_p, x1, x2, x3, h = transfer(theta)
        x, XC, YC = catheter_mesh_1d_total_length(L_x, L_p, x2, x3, h, N_s)
        optimizer.zero_grad()
        out = torch.exp(model(x).squeeze())
        loss = -torch.sum(torch.matmul(out, xx_mask))* L_x/N_s
        loss.backward()
        loss_all.append(loss.item())
        return loss
    
    
    optimizer.step(loss_closure)

    t2 = default_timer()
 
    if ep%1==0:
        
        L_p, x1, x2, x3, h = transfer(theta)
        x, XC, YC = catheter_mesh_1d_total_length(L_x, L_p, x2, x3, h, N_s)
        out = torch.exp(model(x).squeeze())
        loss = -torch.sum(torch.matmul(out, xx_mask))* L_x/N_s
        print(ep, t2 - t1,  "loss = ", loss.item(), "loss_min = ", loss_min.item(),)
 
        if loss < loss_min:
            
            theta_min = theta.detach().numpy()
            loss_min = loss
            
            
            saveplot(theta, out, "movie/design_iter"+ str(plot_id).zfill(4)+".png")
            plot_id += 1
            
        theta_min_perturbed = theta_min + np.random.normal(loc=0.0, scale=1.0, size=len(theta_min))
        
